# IMPORTANTE

Questo file viene utilizzato per provare le chiamate API sfruttando l'infrastruttura di rete fornita da AbitareIn, in modo da sperimentare con l'equipaggiamento messo a disposizione dall'azienda.

Non rappresenta quindi il prodotto finale


### Imports e credenziali ☑️

In [ ]:
import requests
import meraki
import credentials as cr
import time
import os
from pprint import pprint
#import box_sdk_gen
from boxsdk import Client, OAuth2

API_KEY_Meraki = cr.MERAKI_KEY
BASE_URL = "https://api.meraki.com/api/v1/organizations/"
BASE_URL_Centralina = "https://api.ecowitt.net/api/v3/device/"
API_KEY_Centralina = cr.METEO_STATION_API_KEY
APPLICATON_KEY_Centralina = cr.METEO_STATION_APP_KEY

print(API_KEY_Centralina)
print(APPLICATON_KEY_Centralina)

dashboard = meraki.DashboardAPI(api_key=API_KEY_Meraki, output_log=True, print_console=True, log_path='logs/') 
organizations = dashboard.organizations.getOrganizations()

### Ottieni dispositivi dalla centralina meteo ☑️

In [ ]:
def get_weatherstation_device_list():
    params = {
        "application_key": APPLICATON_KEY_Centralina,
        "api_key": API_KEY_Centralina
    }
    response = requests.get(BASE_URL_Centralina + "list", params=params)
    return response.json()

weatherstation_list = get_weatherstation_device_list()


pprint(weatherstation_list)

### Organizzazione ☑️

In [ ]:
ORG_ID = organizations[0]['id']
print (ORG_ID)

### Gestione timestamp ☑️

In [ ]:
from datetime import datetime, timedelta
import pytz

# Usa la data di ieri
rome_tz = pytz.timezone('Europe/Rome')
data = (datetime.now(rome_tz) - timedelta(days=4)).replace(hour=6, minute=0, second=0, microsecond=0)
slot_minutes = (20 - 6) * 60 // 7  # 7 slot tra le 6 e le 20

slot_timestamps = []

#

for i in range(7):
    start_slot = data + timedelta(minutes=i * slot_minutes)
    end_slot = start_slot + timedelta(minutes=slot_minutes - 1, seconds=59)
    slot_timestamps.append({
        "date": start_slot.strftime("%Y-%m-%d"),
        "start": start_slot.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "end": end_slot.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "timeslot": f"{start_slot.strftime('%H:%M:%S')}-{end_slot.strftime('%H:%M:%S')}"
    })
    

# Ciclicamente assegna t0 e t1 ai valori di start e end di ogni slot
for slot in slot_timestamps:
    print(slot)





### Network ☑️

In [ ]:
response_netID = dashboard.organizations.getOrganizationNetworks(ORG_ID)  
pprint(response_netID)
NETWORK_ID = []
for net in response_netID:
    NETWORK_ID.append({
        "name": net['name'],
        "id": net['id']
    })

print("reti salvate \n",NETWORK_ID)

# Print the id of the net with name 'HOM_BIS'
hom_bis_id = next((net['id'] for net in NETWORK_ID if net['name'] == 'HOM_BIS'), None)
print(hom_bis_id)

# Print the position (index) of the 'HOM_BIS' element in NETWORK_ID
hom_bis_index = next((i for i, net in enumerate(NETWORK_ID) if net['name'] == 'HOM_BIS'), None)
print("Position of HOM_BIS in NETWORK_ID:", hom_bis_index)

### Ottenimento seriali di tutti i dispositivi ☑️

In [ ]:
response_serial = dashboard.organizations.getOrganizationDevices(ORG_ID)
pprint(response_serial)

### Categorizzazione seriali dei dispositivi ☑️

In [ ]:
SERIAL = response_serial[0]['serial']
DEVICE_MODEL = response_serial[0]['model']
DEVICE_PRODUCT = response_serial[0]['productType']
print(SERIAL)
DEVICE = {}
i = 0
# To specify a range, use slicing. For example, to get the first 3 serials:
for device in response_serial:
    DEVICE[i] = {
        "NUMBER": i,
        "SERIAL": device['serial'],
        "DEVICE_MODEL": device['model'],
        "DEVICE_PRODUCT": device['productType'],
        "MAC_ADDRESS" : device['mac'],
        "NETWORK_ID": device.get('networkId'),
        "HAS_A_UPLINK" : None,
        "DEVICE_UPLINK" : None
    }
    i = i+1
    print(device)
print(i)





### Processa anche le stazioni meteo ☑️

In [ ]:
def process_weatherstation_devices():
    
    weatherstation_list = get_weatherstation_device_list()

    # Extract the list of weatherstation devices
    weatherstation_data = weatherstation_list.get('data', {})
    weatherstation_devices = weatherstation_data.get('list', [])
    print(weatherstation_devices)
    for device in weatherstation_devices:
        device_name = device.get('name', '')
        # The net name is after 'AI-WS-' in the device name
        if device_name.startswith('AI-WS-'):
            net_name = device_name.replace('AI-WS-', '')
            print(f"Device: {device_name}, Net name: {net_name}")
            # Add weatherstation to DEVICE if net_name matches any network
            matching_net = next((net for net in NETWORK_ID if net['name'] == net_name), None)
            if matching_net:
                new_idx = max(DEVICE.keys()) + 1 if DEVICE else 0
                DEVICE[new_idx] = {
                    "NUMBER": new_idx,
                    "SERIAL": device.get('id', ''),
                    "DEVICE_MODEL": device.get('stationtype', ''),
                    "DEVICE_PRODUCT": "weatherstation",
                    "MAC_ADDRESS": device.get('mac', ''),
                    "NETWORK_ID": matching_net['id'],
                    "HAS_A_UPLINK": True,
                    "DEVICE_UPLINK": device.get('mac', '')
                }
        else:
            print(f"Device name '{device_name}' does not match expected format")



process_weatherstation_devices()

pprint(DEVICE)

### Dispositivi raggiungibili nella rete ☑️

In [ ]:
# Create a list of serials for all devices
serials = [device['serial'] for device in response_serial]

response_uplink = dashboard.organizations.getOrganizationDevicesUplinksAddressesByDevice(ORG_ID, serials = serials)

pprint(response_uplink)

### Viene effettuata una correlazione tra dispositivi raggiungibili nella rete e dispositivi presenti. Dopodiché, vengono stampati i dispositivi rilevati ☑️

In [ ]:
i = 0
UPLINK_DEVICE = {}
def process_uplink_devices():
    #Vengono raggruppati i device con un uplink

    serials = [device["SERIAL"] for device in DEVICE.values() if device.get("DEVICE_PRODUCT") != "weatherstation"]
    response_uplink = dashboard.organizations.getOrganizationDevicesUplinksAddressesByDevice(ORG_ID, serials=serials)

    i = 0
    for device in response_uplink:
        UPLINK_DEVICE[i] = {
        "NUMBER": i,
        "SERIAL": device['serial'],
        "DEVICE_PRODUCT": device['productType'],
        "DEVICE_UPLINK": device['uplinks']
        }
        i += 1

    for idx, device in DEVICE.items():
        serial = device["SERIAL"]
        uplink_info = next((uplink for uplink in UPLINK_DEVICE.values() if uplink["SERIAL"] == serial), None)
        if uplink_info:
            DEVICE[idx]["HAS_A_UPLINK"] = True
            DEVICE[idx]["DEVICE_UPLINK"] = uplink_info["DEVICE_UPLINK"]
        # If device already has uplink True, add to UPLINK_DEVICE
        elif DEVICE[idx]["HAS_A_UPLINK"]:
            UPLINK_DEVICE[idx] = {
                "NUMBER": idx,
                "SERIAL": device['SERIAL'],
                "DEVICE_PRODUCT": device['DEVICE_PRODUCT'],
                "DEVICE_UPLINK": device['DEVICE_UPLINK']
            }
        else:
            DEVICE[idx]["HAS_A_UPLINK"] = False
            DEVICE[idx]["DEVICE_UPLINK"] = None

process_uplink_devices()

pprint(UPLINK_DEVICE)

### Raggruppa i dispositivi a seconda della rete di appartenenza ☑️

In [ ]:
NETWORK_DEVICES = []
for net in NETWORK_ID:
    devices_in_network = [
        {
            "device_name": device.get("DEVICE_MODEL"),
            "device_serial": device.get("SERIAL"),
            "device_category": device.get("DEVICE_PRODUCT"),
            "HAS_A_UPLINK": device.get("HAS_A_UPLINK"),
            "DEVICE_UPLINK": device.get("DEVICE_UPLINK")
        }
        for device in DEVICE.values()
        if device.get("NETWORK_ID") == net["id"]
    ]
    NETWORK_DEVICES.append({
        "Name": net["name"],
        "Id": net["id"],
        "devices": devices_in_network
    })



pprint(NETWORK_DEVICES)


### Vengono individuate i dispositivi a seconda delle loro informazioni ☑️

In [ ]:
def get_device_info(device_type):
    DEVICES = {}

    for net in NETWORK_DEVICES:
        net_id = net["Id"]
        net_name = net["Name"]
        dev = [
            device["device_serial"]
            for device in net["devices"]
            if device.get("device_category", "").lower() == device_type
        ]
        DEVICES[net_name] = {
            "net_id": net_id,
            device_type : dev
        }

    return DEVICES

NET_CAMERAS = {}
NET_CAMERAS = get_device_info("camera")

NET_WEATHERSTATION = get_device_info("weatherstation")


pprint(NET_WEATHERSTATION)


### Scatta una fotografia e la scarica ☑️

In [ ]:
import datetime

# Le reti sono HOM_BIS, THU e B12, vanno inserite in questa riga, al primo get. La funzione in merakiSensors è automatizzata
hom_bis_cameras = NET_CAMERAS.get('THU', {}).get('camera', [])
if not hom_bis_cameras:
    raise ValueError("No cameras found in network.")

for idx, camera_serial in enumerate(hom_bis_cameras):
    response_photo = dashboard.camera.generateDeviceCameraSnapshot(camera_serial)
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    print(f"Camera {camera_serial} response:", response_photo)
    snapshot_url = response_photo['url']
    print(f"Snapshot URL for {camera_serial}:", snapshot_url)

    time.sleep(6)  # Wait for the snapshot to be ready

    download_photo = requests.get(snapshot_url)
    folder = os.path.join("camera_snapshot", "HOM_BIS", f"camera_{idx}")
    os.makedirs(folder, exist_ok=True)

    if 200 <= download_photo.status_code < 300:
        filename = os.path.join(folder, f"camera_{camera_serial}_snapshot_{timestamp}.jpg")
        with open(filename, 'wb') as f:
            f.write(download_photo.content)
        print(f"Photo for camera {camera_serial} downloaded as {filename}")
    else:
        print(f"Failed to download photo for camera {camera_serial}. Status code: {download_photo.status_code}")




### Script per far scattare fotografie ripetute su tutte le reti ☑️

In [ ]:
import time

## Itera per trovare tutte le camere presenti ## 
for net_name, net_info in NET_CAMERAS.items():
    cameras = net_info.get('camera', [])
    if not cameras:
        print(f"No cameras found in network {net_name}.")
        continue

    ## Itera ogni 30 secondi
    while True:

        ## Itera tra tutte le camere trovate nel primo ciclo
        for idx, camera_serial in enumerate(cameras):
            response_photo = dashboard.camera.generateDeviceCameraSnapshot(camera_serial)
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            print(f"Camera {camera_serial} response:", response_photo)
            snapshot_url = response_photo['url']
            print(f"Snapshot URL for {camera_serial}:", snapshot_url)

            time.sleep(6)  # Wait for the snapshot to be ready

            download_photo = requests.get(snapshot_url)
            folder = os.path.join("camera_snapshot", net_name, f"camera_{idx}")
            os.makedirs(folder, exist_ok=True)

            if 200 <= download_photo.status_code < 300:
                filename = os.path.join(folder, f"camera_{camera_serial}_snapshot_{timestamp}.jpg")
                with open(filename, 'wb') as f:
                    f.write(download_photo.content)
                print(f"Photo for camera {camera_serial} downloaded as {filename}")
            else:
                print(f"Failed to download photo for camera {camera_serial}. Status code: {download_photo.status_code}")

        print("Waiting 30 seconds before next cycle...")
        time.sleep(30)


### Upload File su BOX (Al momento non ho né chiavi né account)

In [ ]:
from boxsdk import Client, OAuth2

# Replace with your actual Box credentials
oauth = OAuth2(
    client_id='YOUR_CLIENT_ID',
    client_secret='YOUR_CLIENT_SECRET',
    access_token='YOUR_ACCESS_TOKEN',
)
client = Client(oauth)

# Now you can update the file as intended
client.files.update_file_by_id(
    filename.id, name=filename, description="Updated description"
)

### Vengono estratte info relative al meteo dalle centraline, e riportate all'interno di un CSV☑️

In [ ]:
import os
import csv

def get_network_name(net_id):
    for net in NETWORK_ID:
        if net["id"] == net_id:
            return net["name"]
    return None

def create_CSV_file(DETECTIONS, net_name, category):

    csv_folder = os.path.join("csv", net_name)
    os.makedirs(csv_folder, exist_ok=True)

    # Use the date of the first detection as part of the filename
    if DETECTIONS:
        # Extract date from the first detection slot
        first_date_str = DETECTIONS[0]['date']  # 'YYYY-MM-DD'
        csv_filename = os.path.join(csv_folder, f"{category}_{net_name}_{first_date_str}.csv")

        with open(csv_filename, mode="w", newline="", encoding="utf-8") as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=DETECTIONS[0].keys())
            writer.writeheader()
            writer.writerows(DETECTIONS)
        print(f"CSV file '{csv_filename}' created with aggregated data.")
    else:
        print("No data to write to CSV.")





#i field dove è possibile fare ricerca sono: SERIAL, NUMBER, DEVICE_MODEL, DEVICE_PRODUCT, MAC_ADDRESS, NETWORK_ID, HAS_A_UPLINK, DEVICE_UPLINK
def get_field(id, field):
    for device in DEVICE.values():
        if str(device.get('SERIAL', '')) == str(id):
            return device.get(field, None)
    return None


def get_field_mean(#self,
        field):
    
    field_list = field.get('list', {})

    # Extract only the temperature values from app_temp_list (e.g., '22.5')
    field_values = []
    for v in field_list.values():
        if v not in [None, '', 'null']:
            try:
                field_values.append(float(v))
            except (ValueError, TypeError):
                continue
    if not field_values:
        return None
    mean = sum(field_values) / len(field_values)
    return round(mean, 2)




def SENSE_calculate_weather_conditions(#self,
       network_id, weatherstation_id ):
    

        WEATHER_DETECTIONS = []
        net_name = get_network_name(network_id)

        for slot in slot_timestamps:
            t0 = slot["start"]
            t1 = slot["end"]
            date = slot["date"]
            timeslot = slot["Timeslot"] if "Timeslot" in slot else slot.get("timeslot", None)
            # Ensure t1 > t0
            if t1 <= t0:
                print(f"Skipping slot because t1 ({t1}) is not after t0 ({t0})")
                continue
            weather_info = get_weatherstation_historical_info(weatherstation_id, t0, t1)
            #Estraggo i dati dalla risposta
            data = weather_info.get('data', {})
            if not isinstance(data, dict):
                # Se i dati non sono un dizionario, significa che non ci sono dati disponibili
                print(f"No weather data available for {timeslot} ({date})")
                continue

            outdoor = data.get('outdoor', {})
            app_temp = outdoor.get('app_temp')
            dew_point = outdoor.get('dew_point')
            feels_like = outdoor.get('feels_like')
            humidity = outdoor.get('humidity')
            temperature = outdoor.get('temperature')

            aggregated_data = {
                "date": date,
                "Timeslot": timeslot,
                "app_temp(℃)": get_field_mean(app_temp),    
                "dew_point(℃)": get_field_mean(dew_point),
                "feels_like(℃)": get_field_mean(feels_like),
                "humidity(%)": get_field_mean(humidity),
                "temperature(℃)": get_field_mean(temperature)
            }
            WEATHER_DETECTIONS.append(aggregated_data)

        create_CSV_file(WEATHER_DETECTIONS, net_name, 'weather_detections')


    # Aggregazione dati
    


    #print(f"app_temp: {app_temp}, dew_point: {dew_point}, feels_like: {feels_like}, humidity: {humidity}, temperature: {temperature}")
    #app_temp = weather_info.get


        pprint(WEATHER_DETECTIONS)



    
        


def get_weatherstation_historical_info(id, start, end):

    params = {
        "application_key": APPLICATON_KEY_Centralina,
        "api_key": API_KEY_Centralina,
        "mac": get_field(id, "MAC_ADDRESS"),
        "start_date": start,
        "end_date": end,
        "call_back": "outdoor",
        "temp_unitid": 1,
        "rainfall_unitid": 12,
        "cycle_type": "30min"

    }
    print(start)
    response = requests.get(BASE_URL_Centralina + "history", params=params)
    print("risposta da get_weatherstation_historical_info: \n",response.json())
    return response.json()


# Get the id (SERIAL) of the weatherstation in the network named 'B12'


def get_all_weatherstations_historical_info():
    results = {}
    for net_name, net_info in NET_WEATHERSTATION.items():
        weatherstation_ids = net_info.get('weatherstation', [])
        for ws_id in weatherstation_ids:
            print(f"Getting historical info for weatherstation {ws_id} in network {net_name}")
            info = get_weatherstation_historical_info(ws_id)
            results[(net_name, ws_id)] = info


    return results


def SENSE_calculate_weather_conditions_all_net():
    """
    **Calccola le condizioni meteo in tutte le centraline registrate**
    - 
    """
    for net_name, net_info in NET_WEATHERSTATION.items():
        network_id = net_info.get('net_id')
        weatherstation_ids = net_info.get('weatherstation', [])
        if not weatherstation_ids:
            continue
        for weatherstation_id in weatherstation_ids:
            SENSE_calculate_weather_conditions(network_id, weatherstation_id)


# Example usage:
SENSE_calculate_weather_conditions_all_net()
#all_weatherstation_info = get_all_weatherstations_historical_info()
#pprint(all_weatherstation_info)








### Ottieni dispositivi Wireless Access Point ☑️

In [ ]:
NET_ACCESS_POINTS = {}

for net in NETWORK_DEVICES:
    net_id = net["Id"]
    net_name = net["Name"]
    access_points = [
        device["device_serial"]
        for device in net["devices"]
        if device.get("device_category", "").lower() == "wireless"
    ]
    NET_ACCESS_POINTS[net_name] = {
        "net_id": net_id,
        "access_points": access_points
    }

pprint(NET_ACCESS_POINTS)

### Ottieni Profili wireless della camera ☑️

In [ ]:
from datetime import datetime, timedelta

#network_id 1 funzionante, si dovrebbe montare una logica per montare la netword id da scansionare
network_id = next(net['id'] for net in NETWORK_ID if net['name'] == 'THU')
# Monitoraggio generale, da in uscita tutti i timestamp acquisiti
response_profiles_wireless = dashboard.wireless.getNetworkWirelessClientCountHistory(
    network_id
)

print(response_profiles_wireless)



# Prova con due timestamp
t0 = datetime(2025, 7, 28, 6, 0).isoformat() + "Z"
t1 = datetime(2025, 7, 28, 20, 0).isoformat() + "Z"
#Risoluzione aggiustata per ogni 5 minuti controlla quanti client sono collegati alla rete in un determinato timestamp
resolution = 300

response_profiles_wireless_timestamp = dashboard.wireless.getNetworkWirelessClientCountHistory(
    network_id, timespan = 50400, resolution = resolution
)
# A prescindere dalla risoluzione, si dovrebbero scartare le volte in cui il risultato è none, in modo da ripulire il dataset
print(response_profiles_wireless_timestamp)

client_count = response_profiles_wireless_timestamp[0]['clientCount']


### Statistiche Dettagliate delle connessioni delle persone ☑️

In [ ]:
#t0 = datetime(2025, 7, 14, 6, 0).isoformat() + "Z"
#t1 = datetime(2025, 7, 14, 20, 0).isoformat() + "Z"

# Get the network_id for the network named 'HOM_BIS'


response_network_connection_stats = dashboard.wireless.getNetworkWirelessClientsConnectionStats(
    network_id, timespan = 50400
)
print("RISPOSTA WIRELESS_CLIENT_CONNECTION_STATS\n")
pprint(response_network_connection_stats)

dashboard = meraki.DashboardAPI(API_KEY_Meraki)
response_clients = dashboard.networks.getNetworkClients(
    network_id, total_pages='all', statuses = 'Online', timespan = 50400
)
print("RISPOSTA WIRELESS_CLIENT\n")
pprint(response_clients)



### Overview dei Client Connessi ☑️

In [ ]:
response_client_overview = dashboard.networks.getNetworkClientsOverview(
    network_id, timespan = 50400
)
print(response_client_overview)

### Aggrega i dati appena ottenuti tra tutte le chiamate, in modo da poter essere restituito

versione 1 ✖️

In [ ]:
import os
import csv
from datetime import datetime, timedelta


WIRELESS_DETECTIONS = []
# Crea un set di MAC address da escludere (presenti in DEVICE)
excluded_macs = {device["MAC_ADDRESS"].lower() for device in DEVICE.values()}
#print("MAC addresses da escludere:")
#pprint(excluded_macs)


# Ottieni la data di ieri
oggi = (datetime.now() - timedelta(days=2)).replace(hour=6, minute=0, second=0, microsecond=0)
slot_minutes = (20 - 6) * 60 // 7  # 7 slot tra le 6 e le 20

slot_timestamps = []
for i in range(7):
    start_slot = oggi + timedelta(hours=i * 2)
    end_slot = start_slot + timedelta(hours=1, minutes=59, seconds = 59)
    # Ensure t1 > t0 and use ISO8601 format for Meraki API
    slot_timestamps.append({
        "start": start_slot.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "end": end_slot.strftime("%Y-%m-%dT%H:%M:%SZ")
    })

INIT_TIMESTAMP = slot_timestamps[0]['start']
print(INIT_TIMESTAMP)
END_TIMESTAMP = slot_timestamps[0]['end']

response_clients = dashboard.networks.getNetworkClients(
    NETWORK_ID[2]['id'], total_pages='all'
)
print("RISPOSTA DA NetworkClients\n")
pprint(response_clients)

response_client_overview = dashboard.networks.getNetworkClientsOverview(
    NETWORK_ID[2]['id'], t0=INIT_TIMESTAMP, t1=END_TIMESTAMP
)
print("RISPOSTA DA NetworkClientsOverview")
pprint(response_client_overview)


response_network_connection_stats = dashboard.wireless.getNetworkWirelessClientsConnectionStats(
    NETWORK_ID[2]['id'], t0=INIT_TIMESTAMP, t1=END_TIMESTAMP
)
print("RISPOSTA DA getNetworkWirelessClientsConnectionStats\n")
pprint(response_network_connection_stats)


response_profiles_wireless_timestamp = dashboard.wireless.getNetworkWirelessClientCountHistory(
    NETWORK_ID[2]['id'], t0=INIT_TIMESTAMP, t1=END_TIMESTAMP, resolution = 3600
)
print("RISPOSTA DA NetworkWirelessClientCountHistory\n")
pprint(response_profiles_wireless_timestamp)

for slot in slot_timestamps:
    t0 = slot["start"]
    t1 = slot["end"]

    # Ensure t1 > t0
    if t1 <= t0:
        print(f"Skipping slot because t1 ({t1}) is not after t0 ({t0})")
        continue

    # Aggregazione dati
    aggregated_data = {
        "ts_start": t0,
        "ts_end": t1,
        "device_connected": ""
    }

    response_profiles_wireless_timestamp = dashboard.wireless.getNetworkWirelessClientCountHistory(
        NETWORK_ID[1]['id'], t0=t0, t1=t1
    )
    #print("RISPOSTA DA NetworkWirelessClientCountHistory\n")
    #pprint(response_profiles_wireless_timestamp)

    response_network_connection_stats = dashboard.wireless.getNetworkWirelessClientsConnectionStats(
        NETWORK_ID[1]['id'], t0=t0, t1=t1
    )
    #print("RISPOSTA DA getNetworkWirelessClientsConnectionStats\n")
    #pprint(response_network_connection_stats)

    response_client_overview = dashboard.networks.getNetworkClientsOverview(
        NETWORK_ID[1]['id'], t0=t0, t1=t1
    )
    print("RISPOSTA DA NetworkClientsOverview da"+t0+" a "+t1)
    pprint(response_client_overview)
    #print(aggregated_data)
    # Prendo la chiamata showNetworkDevices e verifico se ci sono dispositivi con mac address presente anche tra i dispositivi personali
    # Ottieni tutti i dispositivi della rete

    # Calcola la media dei client connessi nel periodo specificato
    client_counts = [item['clientCount'] for item in response_profiles_wireless_timestamp if item.get('clientCount') is not None]
    if client_counts:
        avg_clients_connected = sum(client_counts) / len(client_counts)
    else:
        avg_clients_connected = 0
    aggregated_data["device_connected"] = avg_clients_connected
    print(f"Media dispositivi connessi: {avg_clients_connected}")



    macs_in_connection_stats = {stat['mac'].lower() for stat in response_network_connection_stats}
    #print("MAC addresses da response_network_connection_stats:")
    #pprint(macs_in_connection_stats)


    macs_with_usage = {client['mac'].lower() for client in response_clients if client.get('usage', {}).get('total', 0) > 0}
    #print("MAC dei dispositivi con traffico > 0:")
    #pprint(macs_with_usage)



    # Filtra i dispositivi escludendo quelli con MAC address presenti in DEVICE
    # Trova corrispondenze tra clients_mac_addresses e excluded_macs
    matched_macs = {'mac': mac.lower() for mac in macs_with_usage if mac.lower() in excluded_macs}

    #print("Dispositivi di rete :")
    #print(macs_with_usage)
    
    #print("Dispositivi di rete esclusi quelli già presenti in DEVICE:")
    #pprint(matched_macs)


    # incrocio i risultati delle chiamate

    # Trova i dispositivi tra response_clients con traffico totale maggiore di 0
    # Sottrai i MAC address già presenti in matched_macs da quelli con traffico
    remaining_macs = macs_with_usage - set(matched_macs.values())
    # Escludi eventuali elementi nulli da remaining_macs
    remaining_macs = {mac for mac in remaining_macs if mac}
    #print("NUMERO DI MAC RIMASTI \n", remaining_macs)
    aggregated_data["device_connected"] = len(remaining_macs)
    #print("Numero di dispositivi con traffico esclusi quelli già presenti in DEVICE:", aggregated_data["device_connected"])

    #restituisco risultato finale

    #print("Aggregated data:")
    #pprint(aggregated_data)

    WIRELESS_DETECTIONS.append(aggregated_data)

# Convert aggregated_data into a CSV file and save it into the /csv folder

csv_folder = "csv"
os.makedirs(csv_folder, exist_ok=True)

# Use the date of the first detection as part of the filename
if WIRELESS_DETECTIONS:
    # Extract date from the first detection slot
    first_date_str = WIRELESS_DETECTIONS[0]['ts_start'][:10]  # 'YYYY-MM-DD'
    csv_filename = os.path.join(csv_folder, f"wireless_detections_{first_date_str}.csv")

    with open(csv_filename, mode="w", newline="", encoding="utf-8") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=WIRELESS_DETECTIONS[0].keys())
        writer.writeheader()
        writer.writerows(WIRELESS_DETECTIONS)
    print(f"CSV file '{csv_filename}' created with aggregated data.")
else:
    print("No data to write to CSV.")


versione 2 ☑️

In [ ]:
import os
import csv

def get_network_name(net_id):
    for net in NETWORK_ID:
        if net["id"] == net_id:
            return net["name"]
    return None

def calculate_connections(network_id): 

        net_name = get_network_name(network_id)

        WIRELESS_DETECTIONS = []
        number_macs_detected = filter_macs_in_net(network_id)
        

        for slot in slot_timestamps:
            t0 = slot["start"]
            t1 = slot["end"]
            # Ensure t1 > t0
            if t1 <= t0:
                print(f"Skipping slot because t1 ({t1}) is not after t0 ({t0})")
                continue

            # Extract date from the 'start' timestamp
            date_str = slot['start'][:10]  # 'YYYY-MM-DD'
            # Extract only HH:MM:SS from start and end timestamps
            start_time = slot['start'][11:19]  # 'HH:MM:SS'
            end_time = slot['end'][11:19]      # 'HH:MM:SS'
            aggregated_data = {
                "date": date_str,
                "Timeslot": f"{start_time}-{end_time}",
                "DeviceConnected": get_connections_in_timeslot(network_id, t0, t1, number_macs_detected)
            }
            WIRELESS_DETECTIONS.append(aggregated_data)


        create_CSV_file(WIRELESS_DETECTIONS, net_name)

        return WIRELESS_DETECTIONS

    #Funzione che effettua il calcolo complessivo delle wireless detection di tutte le reti
def calculate_connections_all_net():
        for net in NETWORK_ID:
            net_name = net["id"]
            calculate_connections(net_name)

    #Funzione che ritorna il massimo numero di mac address utili individuati all'interno della rete
def filter_macs_in_net(network_id):

        # Mi ricavo tutti i dispositivi individuati a partire dalla giornata lavorativa
        DEVICES_DETECTED = dashboard.networks.getNetworkClients(
            network_id, total_pages = 'all', t0 = slot_timestamps[0]['start']
        )

        # mi filtro i dispositivi che non fanno statistica, ovvero sia
        # 1. I dispositivi che vengono catalogati come centraline meteo (WeatherStation)
        # 2. I dispositivi che non generano traffico di rete
        # 3. I dispositivi che sono all'interno della lista dei dispositivi meraki

        client_macs = {
            client['mac'].lower()
            for client in DEVICES_DETECTED
            if 'mac' in client
            and client.get('description') != 'WeatherStation'
            and client.get('usage', {}).get('total', 0) > 0
        }
        

        filtered_client_macs = client_macs - excluded_macs

        #ritorno il numero di elementi all'interno di filtered_client_macs
        return len(filtered_client_macs)





def get_connections_in_timeslot(network_id, t0,t1, number_macs_detected):


        
        resolution = 300 #300 = ogni 5 minuti. 3600 = ogni ora. Ogni slot 

        connections_in_timeslots = None

        # ASSUNZIONE 1: Considero validi per la mia statistica solo i dispositivi connessi tramite wireless. 
        # Uso getNetworkWirelessClientCountHistory siccome mi restituisce il numero di dispositivi connessi alla rete wireless

        response_profiles_wireless_timestamp = dashboard.wireless.getNetworkWirelessClientCountHistory(
            network_id, t0=t0, t1=t1, resolution = resolution
        )
        print(response_profiles_wireless_timestamp)
        #Calcola la media dei client connessi
        client_counts = [item['clientCount'] for item in response_profiles_wireless_timestamp if item.get('clientCount') is not None]
        if client_counts:
            avg_clients_connected = sum(client_counts) / len(client_counts)
        else:
            avg_clients_connected = 0




        # ASSUNZIONE 2: Considero validi per la mia statistica anche i dispositivi connessi in maniera wired. 
        # In questo caso, utilizzo la chiamata NetworkClientsOverview, in particolare il parametro di clients connected
        # Tuttavia, tra questi valori vanno individuati i dispositivi ininfluenti alla mia statistica, come ad esempio i dispositivi Meraki

        # Per il momento, considero veritiera la prima assunzione
        '''
        response_client_overview = sensors.dashboard.networks.getNetworkClientsOverview(
            network_id, t0=t0, t1=t1
        )

        #considero il minimo tra il numero di dispositivi totali all'interno della risposta in response_clients_overview e il 
        avg_clients_connected = min(response_client_overview.get('counts', {}).get('total', 0), number_macs_detected)

        '''

        #Applico il fattore 0.8: la stima che viene fatta all'interno di un cantiere è che 5 persone possano possedere 6 dispositivi(una persona può avere 1.2 dispositivi)
        connections_in_timeslots = int(avg_clients_connected * 0.8)

        return connections_in_timeslots
        
def create_CSV_file(WIRELESS_DETECTIONS, net_name):

    csv_folder = os.path.join("csv", net_name)
    os.makedirs(csv_folder, exist_ok=True)

    # Use the date of the first detection as part of the filename
    if WIRELESS_DETECTIONS:
        # Extract date from the first detection slot
        first_date_str = WIRELESS_DETECTIONS[0]['date']  # 'YYYY-MM-DD'
        csv_filename = os.path.join(csv_folder, f"wireless_detections_{net_name}_{first_date_str}.csv")

        with open(csv_filename, mode="w", newline="", encoding="utf-8") as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=WIRELESS_DETECTIONS[0].keys())
            writer.writeheader()
            writer.writerows(WIRELESS_DETECTIONS)
        print(f"CSV file '{csv_filename}' created with aggregated data.")
    else:
        print("No data to write to CSV.")
        

WIRELESS_DETECTIONS = calculate_connections(next(net['id'] for net in NETWORK_ID if net['name'] == 'B12'))

# CHIAMATE API DA FINIRE

### Ottieni informazioni mv sense e mqtt broker

In [ ]:
response_mvsense = dashboard.camera.getDeviceCameraSense(
    SERIAL_CAMERA[0]
)
print("RISPOSTA MVSENSE")
print(response_mvsense)

response_mqtt = dashboard.networks.getNetworkMqttBrokers(
    NETWORK_ID[1]['id']
)
print("RISPOSTA MQTT BROKER")
print(response_mqtt)

response = dashboard.camera.getDeviceCameraSenseObjectDetectionModels(
    SERIAL_CAMERA[0]
)
print(response)

### Crea MQTT Broker (da fare una sola volta)

In [ ]:
name = 'MQTT_Broker_1'
host = 'mqtt.merakiabitarein.app'
port = 80

with open("cert.pem", "r") as cert_file:
    ca_certificate = cert_file.read()

try:
    response = dashboard.networks.createNetworkMqttBroker(
        NETWORK_ID[1]['id'],
        name=name,
        host=host,
        port=port,
        security={'mode': 'none'},
        authentication={'username': 'abitarein', 'password': 'abitarein'}
    )
    print(response)
except meraki.APIError as e:
    print("Meraki API Error:", e)
    print("Response:", e.response.text)
except Exception as ex:
    print("General Exception:", ex)

### Imposta MVSense con MQTT Broker (Da fare una sola volta)

In [ ]:

response_set_mvsense = dashboard.camera.updateDeviceCameraSense(
    SERIAL_CAMERA[0], 
    senseEnabled=True, 
    mqttBrokerId='1234', 
    audioDetection={'enabled': False}
)

print(response_set_mvsense)



### Crea un client mqtt che si iscrive al topic relativo alle rilevazioni

In [ ]:
import paho.mqtt.client as mqtt

# Build the topic using the first camera serial
topic = f"/merakimv/{SERIAL_CAMERA[0]}/0"

# Define the MQTT broker and port
broker = "mqtt.merakiabitarein.app"
port = 80

# Define the callback for when a message is received
def on_message(client, userdata, msg):
    print(f"Received message on topic {msg.topic}: {msg.payload.decode()}")

# Create MQTT client and set callback
client = mqtt.Client()
client.on_message = on_message

# Connect to the broker
client.connect(broker, port, 60)

# Subscribe to the topic
client.subscribe(topic)

print(f"Subscribed to topic: {topic}")

# Start the loop to process received messages
client.loop_start()

### Vengono individuate i sensori ☑️

In [ ]:
# Trova il seriale di un dispositivo che è una camera
SERIAL_SENSORS = []
for device in UPLINK_DEVICE.values():
    if device.get("DEVICE_PRODUCT", "").lower() == "sensor":
        SERIAL_SENSORS.append(device["SERIAL"])

print("Sensors serial:", SERIAL_SENSORS)

# CHIAMATE API DI TEST

#### Detection dei movimenti della gru dato un set di foto

In [ ]:
import calcolo_giornata as day
from datetime import datetime, timedelta
import pytz

# Usa la data di ieri
rome_tz = pytz.timezone('Europe/Rome')
giorno_corrente = "2025-09-17"
dati_gru = day.genera_csv_gru(giorno_corrente, "HOM_BIS") or []


In [ ]:
import os

def create_image_list(images_directory):
    file_paths = os.listdir(images_directory)

    for file_path in file_paths:
        split_path = file_path.split('_')
        image_date = split_path[2]
        image_time = split_path[3].split('.')[0]
        full_path = os.path.join(images_directory, file_path)  # path completo
        print(full_path)

create_image_list("camera_snapshot")

#### Aggrega tutti i file CSV in un unico file

# Chiamate API inutilizzate

In [ ]:
#Wireless Detection


url = "https://api.meraki.com/api/v1/networks/"+NETWORK_ID+ "/wireless/clientCountHistory?timespan=360"

payload = None

headers = {
    "Authorization": "Bearer "+API_KEY,
    "Accept": "application/json"
}

response = requests.request('GET', url, headers=headers, data = payload)

data = response.json()
pprint(data)



response_channel_utilization = dashboard.wireless.getNetworkWirelessChannelUtilizationHistory(
    NETWORK_ID, t0 = t0, t1 = t1, deviceSerial = SERIAL_WIRELESS
)

pprint("RISPOSTA CHANNEL_UTILIZATION \n", response_channel_utilization)


response_profiles = dashboard.camera.getNetworkCameraWirelessProfiles(
    NETWORK_ID
)
print("RISPOSTA WIRELESS_PROFILES \n")
pprint(response_profiles)

response_profiles_serial = dashboard.camera.getDeviceCameraWirelessProfiles(
    SERIAL_CAMERA
)
print("RISPOSTA SERIAL_PROFILES \n")
pprint(response_profiles_serial)


response_marshal = dashboard.wireless.getNetworkWirelessAirMarshal(
    NETWORK_ID
)

print(response_marshal)

response_clients_bt = dashboard.networks.getNetworkBluetoothClients(
    NETWORK_ID, total_pages='all'
)
print("RISPOSTA BLUETOOTH_CLIENT\n")
pprint(response_clients_bt)

### Ottieni video link della camera

In [ ]:
# Define the start timestamp at 15:30:00
start_time = datetime(2025, 7, 7, 15, 30, 0)
# Define the end timestamp at 15:31:00
end_time = start_time + timedelta(minutes=1)

t_start = start_time.isoformat() + "Z"
t_end = end_time.isoformat() + "Z"

print("Start timestamp:", t_start)
print("End timestamp:", t_end)


response_video = dashboard.camera.getDeviceCameraVideoLink(
    SERIAL_CAMERA[1]
)

print(response_video)

### Ottieni l'ID dei boundaries che la camera è in grado di vedere

In [ ]:
url_boundary = "https://api.meraki.com/api/v1/organizations/"+ORG_ID+"/camera/boundaries/areas/byDevice"
headers = {
    "Content-Type": "application/json",
    "X-Cisco-Meraki-API-Key": API_KEY
}
response = requests.get(url_boundary, headers=headers)
print(response)

### Verifica quante detection in un intervallo di tempo

In [ ]:
url = "https://api.meraki.com/api/v1/organizations/"+ORG_ID+"camera/detections/history/byBoundary/byInterval"
querystring = {"boundaryIds[]":"682858293500052355","ranges[][startTime]":t0,"ranges[][endTime]":t1,"ranges[][interval]":"7200"}

headers = {
    "Content-Type": "application/json",
    "X-Cisco-Meraki-API-Key": cr.API_KEY
}
response = requests.get(url, headers=headers, params=querystring)
print(response)